### Install packages

In [1]:
!pip install together
!pip install git+https://github.com/huggingface/transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 kB 4.8 MB/s eta 0:00:00
DEPRECATION: beakerx-base 2.2.0 has a non-standard dependency specifier ipywidgets<8pandas,>=7.5.1. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of beakerx-base or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-ejxk0u37
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-ejxk0u37
  Resolved https://github.com/huggingface/transformers to commit cc3e4781854a52cf090ffde28d884a527dab6708
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 21.5 MB/s eta 0:00:00 0:0

In [2]:
!pip show together

Name: together
Version: 0.2.6
Summary: Python client for Together's Cloud Platform!
Home-page: https://github.com/togethercomputer/together
Author: Together AI
Author-email: support@together.ai
License: Apache-2.0
Location: /opt/conda/lib/python3.9/site-packages
Requires: requests, sseclient-py, tabulate, tqdm, typer
Required-by: 


### Import packages

In [3]:
from transformers import AutoTokenizer, LlamaForCausalLM
#from human_eval.data import write_jsonl, read_problems
import tqdm
import time

In [4]:
from pathlib import Path
import os

In [5]:
import sys
sys.path.append("/opt/conda/lib/python3.9/site-packages")
import together

### LLM Server Checkpoint load

In [7]:
# model = LlamaForCausalLM.from_pretrained(model_path, device_map="auto")
# tokenizer = AutoTokenizer.from_pretrained(model_path, padding_side='left')

### Phind_LLM function

In [8]:
def phind_LLM(model_path, prompt_template):
    
    model = LlamaForCausalLM.from_pretrained(model_path, device_map="auto")
    tokenizer = AutoTokenizer.from_pretrained(model_path, padding_side='left')
    model_path = "Phind/Phind-CodeLlama-34B-v2"
    
    tokenizer = AutoTokenizer.from_pretrained(model_path, padding_side='left')
    start_time =time.time()
    inputs = tokenizer.encode_plus(prompt_template, return_tensors="pt", truncation=True, max_length=1024, padding="max_length")

    # Input id and generation mask
    input_ids = inputs.input_ids
    attention_mask = inputs.attention_mask

    # output generation
    print("Phind-CodeLlama-34B-v2 generation in process")
    generate_ids = model.generate(input_ids, attention_mask=attention_mask, max_new_tokens=1024, do_sample=False, top_p=0.75, top_k=60, temperature=0.1)

    # Decoding output
    generated_code = tokenizer.decode(generate_ids[0], skip_special_tokens=True)
    generated_code = generated_code.replace(prompt_template, "").split("\n\n\n")[0]

    end_time = time.time()
    execution_time - end_time -start_time
    print(generated_code)
    return generated_code, execution_time

### together.ai API implementation

In [9]:
together.api_key = "e03a7067bb512668df6dc48cffc1da1b04ff24b3639222546cffec115fd1e2fb"

#### model list

In [10]:
# see available models
model_list = together.Models.list()

print(f"{len(model_list)} models available")

# print the first 10 models on the menu
model_names = [model_dict['name'] for model_dict in model_list]
model_names

89 models available


['Austism/chronos-hermes-13b',
 'EleutherAI/llemma_7b',
 'EleutherAI/pythia-12b-v0',
 'EleutherAI/pythia-1b-v0',
 'EleutherAI/pythia-2.8b-v0',
 'EleutherAI/pythia-6.9b',
 'Gryphe/MythoMax-L2-13b',
 'HuggingFaceH4/starchat-alpha',
 'NousResearch/Nous-Hermes-13b',
 'NousResearch/Nous-Hermes-Llama2-13b',
 'NousResearch/Nous-Hermes-Llama2-70b',
 'NousResearch/Nous-Hermes-llama-2-7b',
 'NumbersStation/nsql-llama-2-7B',
 'Open-Orca/Mistral-7B-OpenOrca',
 'OpenAssistant/llama2-70b-oasst-sft-v10',
 'OpenAssistant/oasst-sft-4-pythia-12b-epoch-3.5',
 'OpenAssistant/stablelm-7b-sft-v7-epoch-3',
 'Phind/Phind-CodeLlama-34B-Python-v1',
 'Phind/Phind-CodeLlama-34B-v2',
 'SG161222/Realistic_Vision_V3.0_VAE',
 'WizardLM/WizardCoder-15B-V1.0',
 'WizardLM/WizardCoder-Python-34B-V1.0',
 'WizardLM/WizardLM-70B-V1.0',
 'bigcode/starcoder',
 'databricks/dolly-v2-12b',
 'databricks/dolly-v2-3b',
 'databricks/dolly-v2-7b',
 'defog/sqlcoder',
 'garage-bAInd/Platypus2-70B-instruct',
 'huggyllama/llama-13b',
 'h

In [11]:
code_llm_dict = {
1 : 'Phind/Phind-CodeLlama-34B-v2',
2 : 'Phind/Phind-CodeLlama-34B-Python-v1',
3 : 'WizardLM/WizardCoder-Python-34B-V1.0',
4 : 'WizardLM/WizardCoder-15B-V1.0',
5 : 'togethercomputer/CodeLlama-13b-Instruct',
6 : 'togethercomputer/CodeLlama-13b-Python',
7 : 'togethercomputer/CodeLlama-13b',
8 : 'togethercomputer/CodeLlama-34b-Instruct',
9 : 'togethercomputer/CodeLlama-34b-Python',
10 : 'togethercomputer/CodeLlama-34b',
11 :'togethercomputer/CodeLlama-7b-Instruct',
12 : 'togethercomputer/CodeLlama-7b-Python',
13 : 'togethercomputer/CodeLlama-7b',
}
# print(code_llm_dict)

#### together.ai API Code

In [12]:
def together_api(input_promt):
    code_llm_dict = {
    1: 'Phind/Phind-CodeLlama-34B-v2',
    2: 'Phind/Phind-CodeLlama-34B-Python-v1',
    3: 'WizardLM/WizardCoder-Python-34B-V1.0',
    4: 'WizardLM/WizardCoder-15B-V1.0',
    5: 'togethercomputer/CodeLlama-13b-Instruct',
    6: 'togethercomputer/CodeLlama-13b-Python',
    7: 'togethercomputer/CodeLlama-13b',
    8: 'togethercomputer/CodeLlama-34b-Instruct',
    9: 'togethercomputer/CodeLlama-34b-Python',
    10: 'togethercomputer/CodeLlama-34b',
    11: 'togethercomputer/CodeLlama-7b-Instruct',
    12: 'togethercomputer/CodeLlama-7b-Python',
    13: 'togethercomputer/CodeLlama-7b',
        }
    # Provide user options to select
    print("Available options:")
    for key, value in code_llm_dict.items():
        print(f"{key}: {value}")

    # Get user input
    selected_option = int(input("Select an option  "))

    # Check if the selected option is valid
    if selected_option in code_llm_dict:
        selected_LLM_model = code_llm_dict[selected_option]
        print(f"You selected: {selected_LLM_model}")
    else:
        print("Invalid option. Please select a valid option.")

    start_time = time.time()
    print(f"{selected_LLM_model}: generation in process")
    output = together.Complete.create(
    prompt = input_promt, 
    model = selected_LLM_model, 
    max_tokens = 1024,
    temperature = 0.1,
    top_k = 60,
    top_p = 0.75,
    repetition_penalty = 1.1
    #stop = ['<human>', '\n\n']
    )
    end_time = time.time()
    execution_time = end_time - start_time
    generated_code = output['output']['choices'][0]['text']

    # print generated text
    # print(output['prompt'][0]+output['output']['choices'][0]['text'])
    # print(output['output']['choices'][0]['text'])
    return generated_code, execution_time


###  prompt generation from file

In [13]:
def promt_generation():
    # PolyCoder_Data_Collection/Code-LMs/Data/code_generation_script/axle_projects/filepattern/src/filepattern/cpp/include/filepattern.h
    instruction = " Generate C++ bindings ('.cpp' file) with the Pybind11 module for a C++ class and provide only the output code without any explanation for :"
    
    # instruction = """ Complete the cmake file library name is 'backend' and target_compile_definitions
    # file_path = 'src/filepattern/cpp/bindings.cpp' :
    # if(BUILD_PYTHON_LIB)
    #     find_package(pybind11 CONFIG REQUIRED) """

    # Initialize a dictionary to store file contents with filenames as keys
    file_contents = {}

    # Prompt the user for the number of file paths they want to input
    num_paths = int(input("Enter the number of file paths: "))

    # Loop to gather and read file contents into variables
    for i in range(num_paths):
        file_path = input(f"Enter the file path {i + 1}: ")
        file_path = Path(file_path)
        normalized_path = os.path.normpath(file_path)

    # Read the file content into a variable
    with open(normalized_path, 'r', encoding="utf-8") as source_file:
        file_contents[f'file_{i + 1}'] = source_file.read()

    # Create a prompt_template with the instruction and file contents
    prompt_template = instruction
    for key in file_contents:
        prompt_template += f" {file_contents[key]}"

    # print(f"Generated Prompt Template: {prompt_template}")

    llm_promt = f'''{prompt_template}'''
    return llm_promt

### Write_Output funtion

In [14]:
def write_data(generated_binding,save_path):
    # Write the content to the .cpp file
    with open(save_path, 'w', encoding='utf-8') as cpp_file:
        cpp_file.write(generated_binding)

## LLM main funtion

In [19]:
if __name__ == '__main__':

    print("Select an option:")
    print("1. Generate code from SCB NCAT LLM server (Internal server)")
    print("2. Generate code from together.ai API")

    # Get user input
    user_choice = input("Enter the number of your choice: ")
    generated_promt = promt_generation()
    # print(generated_promt)
    # save_path = 'bindings.cpp'
    save_path = 'cmake.txt'


    if user_choice == '1':
        print("Selected : SCB NCAT LLM server")

        generated_binding,execution_time = phind_LLM(model_path, generated_promt)

    elif user_choice == '2':
        # Option 2: Code generation from together.ai API
        print("Code generation from together.ai API")
        generated_binding, execution_time = together_api(generated_promt)

    else:
        print("Invalid option. Please select a valid option (1 or 2).")
    write_data (generated_binding,save_path)

    print(f" Model code generation time: {execution_time}")

Select an option:
1. Generate code from SCB NCAT LLM server (Internal server)
2. Generate code from together.ai API


Enter the number of your choice:  1
Enter the number of file paths:  1
Enter the file path 1:  work/axle_projects/nyxus/src/nyx/environment.h


FileNotFoundError: [Errno 2] No such file or directory: 'work/axle_projects/nyxus/src/nyx/environment.h'

In [ ]:
PolyCoder_Data_Collection/Code-LMs/Data/code_generation_script/axle_projects/filepattern/src/filepattern/cpp/include/filepattern.h
../axle_projects/nyxus/src/nyx/environment.h